In [10]:
import os
import paramiko
from scp import SCPClient
from datetime import datetime
import time
import ctypes
from ctypes import wintypes
import sys
from typing import Dict, Any, Optional

# --- WINDOWS SLEEP PREVENTION ---

# Constants from winnt.h
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001
ES_DISPLAY_REQUIRED = 0x00000002

def prevent_sleep():
    """Enable system sleep prevention."""
    # Only run on Windows
    if sys.platform.startswith('win'):
        ctypes.windll.kernel32.SetThreadExecutionState(
            ES_CONTINUOUS | ES_SYSTEM_REQUIRED | ES_DISPLAY_REQUIRED
        )

def allow_sleep():
    """Allow sleep again."""
    # Only run on Windows
    if sys.platform.startswith('win'):
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)

# --- HELPER FUNCTION ---

def file_exists_in_subdirs(root_dir: str, filename: str) -> bool:
    """Recursively search for filename in root_dir and subdirectories."""
    # This assumes the file on the controller is the exact name we are saving locally
    for dirpath, _, filenames in os.walk(root_dir):
        if filename in filenames:
            return True
    return False

# --- MAIN DOWNLOAD FUNCTION ---

def download_remote_files(
    device_config: Dict[str, Any],
    mode: str = 'once',  # 'once' or 'continuous'
    t_hour: float = 0.25 # Time between runs for 'continuous' mode (in hours)
) -> None:
    """
    Connects to a remote device via SSH/SCP and downloads files.

    Args:
        device_config: Dictionary containing 'host', 'port', 'user', 'password', 
                       'remote_folder', and 'local_root_folder'.
        mode: 'once' for a single download, 'continuous' to loop.
        t_hour: Sleep duration in hours for 'continuous' mode.
    """
    
    # Unpack configuration
    REMOTE_HOST = device_config['host']
    REMOTE_PORT = device_config['port']
    USERNAME = device_config['user']
    PASSWORD = device_config['password']
    REMOTE_FOLDER = device_config['remote_folder']
    LOCAL_ROOT_FOLDER = device_config['local_root_folder']
    
    # Calculate sleep duration in seconds
    SLEEP_DURATION = t_hour * 60 * 60 

    def _do_download():
        """The core logic for connecting and downloading."""
        print(f"[{datetime.now()}] Starting SCP download job from {REMOTE_HOST}...")

        if not os.path.exists(LOCAL_ROOT_FOLDER):
            print(f"Creating local folder: {LOCAL_ROOT_FOLDER}")
            os.makedirs(LOCAL_ROOT_FOLDER)

        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        try:
            # 1. Connect
            print(f"[{datetime.now()}] Attempting to connect to {USERNAME}@{REMOTE_HOST}:{REMOTE_PORT}")
            ssh.connect(REMOTE_HOST, port=REMOTE_PORT, username=USERNAME, password=PASSWORD)

            # 2. List Files - FIX APPLIED HERE
            # The 'ls -p | grep -v /' command lists files but excludes directories (which end in /).
            # This is key for devices that may have directories in the data folder.
            list_command = f'ls -p {REMOTE_FOLDER} | grep -v /'
            stdin, stdout, stderr = ssh.exec_command(list_command)
            
            # --- FIX: Read stdout once ---
            output = stdout.read().decode().strip()
            
            files = output.splitlines()

            if not files:
                print(f"[{datetime.now()}] No files found in remote directory: {REMOTE_FOLDER}")
                return

            # 3. Download Files
            with SCPClient(ssh.get_transport()) as scp:
                for file in files:
                    remote_path = f"{REMOTE_FOLDER}/{file}"
                    # Note: We use os.path.join for the local path for platform independence
                    local_target_path = os.path.join(LOCAL_ROOT_FOLDER, file)
                    
                    if file_exists_in_subdirs(LOCAL_ROOT_FOLDER, file):
                        print(f"Skipping (already exists locally): {file}")
                    else:
                        print(f"Downloading: {file}")
                        # scp.get handles file existence checks on the remote side gracefully
                        scp.get(remote_path, local_target_path)
            
            print(f"[{datetime.now()}] Download job completed successfully.")

        except paramiko.AuthenticationException:
            print(f"[{datetime.now()}] Error: Authentication failed for {USERNAME}@{REMOTE_HOST}")
        except paramiko.SSHException as e:
            print(f"[{datetime.now()}] Error: SSH connection or command failed: {e}")
        except Exception as e:
            print(f"[{datetime.now()}] General Error during download: {e}")
        finally:
            ssh.close()
            
    # --- Mode Execution Logic ---
    
    if mode == 'once':
        _do_download()
        
    elif mode == 'continuous':
        print(f"Starting continuous SCP downloader for {REMOTE_HOST}...")
        print("Press Ctrl+C to stop.")
        
        # Prevent system sleep while script is running
        prevent_sleep()
        
        try:
            _do_download() # Initial run
            
            while True:
                print(f"[{datetime.now()}] Waiting {t_hour} hours ({int(SLEEP_DURATION)} seconds) before next run...")
                time.sleep(SLEEP_DURATION)
                print(f"[{datetime.now()}] Waking up for next download cycle.")
                _do_download()
                
        except KeyboardInterrupt:
            print("\nScript stopped by user.")
        finally:
            allow_sleep()
            
    else:
        print(f"Error: Invalid mode '{mode}'. Use 'once' or 'continuous'.")


# --- CONFIGURATION DICTIONARIES ---

# Traffic Controller Configuration (Banks)
BANKS_CONTROLLER_CONFIG = {
    'host': "10.37.23.200",
    'port': 22,
    'user': "econolite",
    'password': "ecpi2ecpi",
    'remote_folder': "/opt/econolite/set1",
    'local_root_folder' : r"C:/Users/rhansen/Documents/Python/SPMs/Intersections/201 SH-55 & Banks-Lowman/Data/DATZ/"
}

# EVO Radar Device Configuration (Banks)
BANKS_EVO_CONFIG = {
    'host': "10.37.23.201", #evo
    'port': 22,
    'user': "evo",
    'password': "root",
    'remote_folder': "/home/evo/Documents/SPM_Data_All",
    'local_root_folder' : r"C:/Users/rhansen/Documents/Python/SPMs/Intersections/201 SH-55 & Banks-Lowman/Data/DATZ/"
}

MOSCOW_CONFIGS = [
#    {
#        'host': "10.37.2.66", #Washington & 3rd
#        'port': 22,
#        'user': "econolite",
#        'password': "ecpi2ecpi",
#        'remote_folder': "/opt/econolite/set1",
#        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2066_US-95_Washington_and_3rd_St/raw_data/"
#    },
#    {
#        'host': "10.37.2.67", #Washington & 6th
#        'port': 22,
#        'user': "econolite",
#        'password': "ecpi2ecpi",
#        'remote_folder': "/opt/econolite/set1",
#        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2067_US-95_Washington_and_6th_St/raw_data/"
#    },
#    {
#        'host': "10.37.2.76", #Jackson & 6th
#        'port': 22,
#        'user': "econolite",
#        'password': "ecpi2ecpi",
#        'remote_folder': "/opt/econolite/set1",
#        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2076_US-95_Jackson_and_6th/raw_data/"
#    },
    {
        'host': "10.37.2.68", #US-95 & SH-8
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2068_US-95_and_SH-8/raw_data/"
    },
    {
        'host': "10.37.2.69", #US-95 & Sweet Ave
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2069_US-95_and_Sweet_Ave/raw_data/"
    },
    {
        'host': "10.37.2.70", #US-95 & Palouse River Dr
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2070_US-95_and_Palouse_River_Dr/raw_data/"
    },
    {
        'host': "10.37.2.72", #US-95 & D St
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2072_US-95_and_D_St/raw_data/"
    },
    {
        'host': "10.37.2.79", #SH-8 & Line St
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2079_SH-8_and_Line_St/raw_data/"
    },
    {
        'host': "10.37.2.80", #SH-8 & Peterson
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2080_SH-8_and_Peterson/raw_data/"
    },
    {
        'host': "10.37.2.81", #SH-8 & Perimeter
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2081_SH-8_and_Perimeter/raw_data/"
    },
    {
        'host': "10.37.2.82", #SH-8 & Warbonnet
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2082_SH-8_and_Warbonnet/raw_data/"
    },
    {
        'host': "10.37.64.21", #US-12 & 3rd St
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2121_US-12_and_3rd_St/raw_data/"
    },
    {
        'host': "10.37.64.31", #US-12 & 21st
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2131_US-12_and_21st/raw_data/"
    },
    {
        'host': "10.37.64.41", #US-12 & 18th
        'port': 22,
        'user': "econolite",
        'password': "ecpi2ecpi",
        'remote_folder': "/opt/econolite/set1",
        'local_root_folder' : r"C:/Users/rhansen/Documents/Python/pyatspm/intersections/2141_US-12_and_18th/raw_data/"
    }
]


In [9]:
for config in MOSCOW_CONFIGS:
    download_remote_files(config, mode='once')

[2026-02-17 16:44:30.074372] Starting SCP download job from 10.37.2.68...
[2026-02-17 16:44:30.075373] Attempting to connect to econolite@10.37.2.68:22
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1030.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1045.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1100.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1115.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1130.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1145.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1200.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1215.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1230.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1245.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_02_16_1300.datZ
Skipping (already exists locally): ECON_10.37.2.68_2026_

In [5]:

# --- EXAMPLE USAGE ---

# Choose the device you want to download from
#CURRENT_CONFIG = BANKS_CONTROLLER_CONFIG
CURRENT_CONFIG = BANKS_EVO_CONFIG

# Choose the mode and time interval
DOWNLOAD_MODE = 'once'  # Options: 'once', 'continuous'
DOWNLOAD_INTERVAL_HOURS = 0.25 # Run every 15 minutes for continuous mode

# Execute the function
if __name__ == "__main__":
    download_remote_files(
        device_config=CURRENT_CONFIG, 
        mode=DOWNLOAD_MODE, 
        t_hour=DOWNLOAD_INTERVAL_HOURS
    )

[2026-02-17 00:42:44.902985] Starting SCP download job from 10.37.23.201...
[2026-02-17 00:42:44.902985] Attempting to connect to evo@10.37.23.201:22
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0000.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0015.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0030.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0045.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0100.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0115.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0130.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0145.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0200.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0215.datZ
Skipping (already exists locally): ECON_10.37.23.200_2026_01_01_0230.datZ
Skipping (already exists locally): E